<a href="https://colab.research.google.com/github/LuigiSigillo/CartoonGAN/blob/main/CartoonGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
from google.colab import drive
import os
import json
import re
import cv2
import numpy as np
from PIL import Image, ImageFilter
import sys
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')
#!unzip /content/drive/My\ Drive/NN/spirited_away.zip -d /content/drive/My\ Drive/NN/
!ls /content/drive/My\ Drive/NN/

# 1 - Image Preprocessing

### 1.1 - Resizing images


In [ ]:
def resize(path):
    for item in os.listdir(path):
            im = Image.open(path+item)
            f, e = os.path.splitext(item)
            imResize = im.resize((256,256), Image.ANTIALIAS)
            print(f)
            imResize.save(path+"_resized/" + f + ' resized.jpg', 'JPEG', quality=90)

resize('/content/drive/My Drive/NN/your_name')

### 1.2 Apply canny, dilate edge and gaussian

In [ ]:
def edge_smoothing(cartoon_images_filename, smoothed_images_filename):
    print("Edge-smoothing of ", cartoon_images_filename)
    origin = cv2.imread(cartoon_images_filename)
    edges = createEdgesOverlay(origin)
    result = overlayEdges(edges, origin)
    #show_images(origin, edges, result)
    result.save(smoothed_images_filename, "JPEG")

def overlayEdges(edges, origin):
    background = transformFromCV2ToPillowImageFormat(origin)
    background.paste(edges, (0, 0), edges)
    background = background.convert("RGB")
    return background

def transformFromCV2ToPillowImageFormat(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGBA)
    return Image.fromarray(img)

def createEdgesOverlay(origin):
    edges = cv2.Canny(origin, 30, 300, 3) 
    edges = cv2.dilate(edges, (3, 3))
    edges = cv2.bitwise_not(edges)
    edges = transformFromCV2ToPillowImageFormat(edges)
    makeWhiteBackgroundTransparent(edges)
    edges = edges.filter(ImageFilter.GaussianBlur) #do blurring here because doing it before making background transparent results in white halo

    return edges

def makeWhiteBackgroundTransparent(img):
    datas = img.getdata()
    newData = []
    for item in datas:
        if item[0] == 255 and item[1] == 255 and item[2] == 255:
            newData.append((255, 255, 255, 0))
        else:
            newData.append(item)
    img.putdata(newData)

def show_images(img,edges,result):
    plt.subplot(131),plt.imshow(img)
    plt.title('Original Image'), plt.xticks([]), plt.yticks([])

    plt.subplot(132),plt.imshow(edges)
    plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
    
    plt.subplot(133),plt.imshow(result)
    plt.title('Result Image'), plt.xticks([]), plt.yticks([])

    plt.show()
    



path_resized = "/content/drive/My Drive/NN/spirited_away_resized/"
path_smoothed = "/content/drive/My Drive/NN/spirited_away_resized_smoothed/"


for filename in os.listdir(path_resized):
  #filename='scene43626 resized.jpg'
  f = filename.split(" ")[0] + " smoothed"
  cartoon_images_filename = path_resized + filename
  smoothed_images_filename = path_smoothed + f
  edge_smoothing(cartoon_images_filename, smoothed_images_filename)



# GAN

## Load VGG19 (transfer learning)

In [ ]:
from keras import applications
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers

#TODO
def my_loss_fn(y_true, y_pred):
    squared_difference = tf.square(y_true - y_pred)
    return tf.reduce_mean(squared_difference, axis=-1)


def load_backbone_net(input_shape):
    
    # define input tensor
    input0 = Input(shape=input_shape)

    # load a pretrained model on imagenet without the final dense layer
    feature_extractor = applications.vgg19.VGG19(include_top=False, weights='imagenet', input_tensor=input0)
    
    feature_extractor = feature_extractor.output
    feature_extractor = Model(inputs=input0, outputs=feature_extractor)
    optimizer = 'adam' #alternative 'SGD'

    feature_extractor.compile(loss=my_loss_fn, optimizer=optimizer, metrics=['accuracy'])

    return feature_extractor


def transferNet(feature_extractor, output_layer_name, trainable_layers):
    
    # get the original input layer tensor
    input_t = feature_extractor.get_layer(index=0).input

    # set the feture extractor layers as non-trainable
    for idx,layer in enumerate(feature_extractor.layers):
      if layer.name in trainable_layers:
        layer.trainable = True
      else:
        layer.trainable = False

    # get the output tensor from a layer of the feature extractor
    output_extractor = feature_extractor.get_layer(name = output_layer_name).output
    
    #output_extractor = MaxPooling2D(pool_size=(4,4))(output_extractor)

    # flat the output of a Conv layer
    flatten = Flatten()(output_extractor) 
    flatten_norm = BatchNormalization()(flatten)

    # add a Dense layer
    dense = Dropout(0.4)(flatten_norm)
    dense = Dense(200, activation='relu')(dense)
    dense = BatchNormalization()(dense)
    
    # add a Dense layer
    dense = Dropout(0.4)(dense)
    dense = Dense(100, activation='relu')(dense)
    dense = BatchNormalization()(dense)    

    model = Model(inputs=input_t, outputs=dense, name="transferNet")
    
    optimizer = 'adam' #alternative 'SGD'
    model.compile(loss=my_loss_fn, optimizer=optimizer, metrics=['accuracy'])

    return model


input_shape =(256,256,3)
# load the pre-trained model
feature_extractor = load_backbone_net(input_shape)
feature_extractor.summary()


# choose the layer from which you can get the features 
name_output_extractor = "block4_conv4" #ultimo
trainable_layers = ["block4_conv3","block4_conv2"] #penultimo

# build the transfer model
transfer_model = transferNet(feature_extractor, name_output_extractor, trainable_layers)
transfer_model.summary()



## Generator

In [52]:
from keras import applications
from keras import layers
from keras.models import Model, Input
from keras.models import Sequential
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from keras import regularizers
from keras import optimizers


def residual_block(y, nb_channels):
    shortcut = y
    print("c")
    # down-sampling is performed with a stride of 1
    y = Conv2D(nb_channels, kernel_size=3, strides=1, padding='valid',activation="relu")(y)
    y = ZeroPadding2D(padding=(1, 1))(y)
    y = Conv2D(nb_channels, kernel_size=3, strides=1, padding='valid',activation="relu")(y)
    y = ZeroPadding2D(padding=(1, 1))(y)
    y = BatchNormalization()(y)
    y = BatchNormalization()(y)

    y = layers.add([shortcut, y])
    return y


def generator():
    model = Sequential()

    model.add(Conv2D(filters=64, kernel_size=7, strides=1, padding='valid',activation="relu"))
    model.add(ZeroPadding2D(padding=(3, 3)))
    model.add(BatchNormalization())
    
    #### Down-Convolution
    
    #k3 n128 s2
    model.add(Conv2D(filters=128, kernel_size=3, strides=2, padding='valid',activation="relu"))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(BatchNormalization())
    #k3 n128 s1
    model.add(Conv2D(filters=128, kernel_size=3, strides=1, padding='valid',activation="relu"))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(BatchNormalization())

    #k3 n256 s2
    model.add(Conv2D(filters=256, kernel_size=3, strides=2, padding='valid',activation="relu"))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(BatchNormalization())
    #k3 n256 s1
    model.add(Conv2D(filters=256, kernel_size=3, strides=1, padding='valid',activation="relu", name="test"))
    model.add(ZeroPadding2D(padding=(1, 1)))
    model.add(BatchNormalization())

    
    # residual blocks
    for i in range(8):#number of res blocks
        model.add(residual_block(,256))
    
    
    # Up-convolution
    model.add(Conv2DTranspose(filters=128, kernel_size=3, strides=2, padding='valid',output_padding=1,activation='relu'))
    model.add(ZeroPadding2D(padding=(1, 1)))
    
    model.add(Conv2DTranspose(filters=128, kernel_size=3, strides=1, padding='valid',activation='relu'))
    model.add(ZeroPadding2D(padding=(1, 1)))
    
    model.add(keras.layers.BatchNormalization())
    #######################
    model.add(Conv2DTranspose(filters=64, kernel_size=3, strides=2, padding='valid',output_padding=1,activation='relu'))
    model.add(ZeroPadding2D(padding=(1, 1)))
    
    model.add(Conv2DTranspose(filters=64, kernel_size=3, strides=1, padding='valid',activation='relu'))
    model.add(ZeroPadding2D(padding=(1, 1)))

    model.add(BatchNormalization())

    ###
    model.add(Conv2D(filters=3, kernel_size=7, strides=1, padding='valid',activation="sigmoid"))
    model.add(ZeroPadding2D(padding=(3, 3)))

    return model

model = generator()

c


TypeError: ignored

## Utils

###Save model

In [ ]:
models_dir = "/content/drive/My Drive/NN/"

def savemodel(model,problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    model.save(filename)
    print("\nModel saved successfully on file %s\n" %filename)

# Save the model MWI-Dataset-1.1_models
savemodel(model,'Twd_model_31_epochs_2000_images')


### Load model

In [ ]:
from keras.models import load_model

models_dir = "/content/drive/My Drive/NN/"
model_name = "towards_model_100_epochs_2000_images_acc44_256"

def loadmodel(problem):
    filename = os.path.join(models_dir, '%s.h5' %problem)
    try:
        model = load_model(filename)
        print("\nModel loaded successfully from file %s\n" %filename)
    except OSError:    
        print("\nModel file %s not found!!!\n" %filename)
        model = None
    return model

model = loadmodel(model_name)